In [ ]:
#Good read on sending messages to event grid and converting to CSV
# https://docs.microsoft.com/en-us/azure/event-hubs/get-started-capture-python-v2


In [ ]:
import pandas as pd
df = pd.read_csv('4D-Lung_studies.csv')
len(df)

In [ ]:
df = pd.read_csv('4D-Lung_studies.csv')

In [ ]:
len(df)

In [ ]:
stuff = df.iterrows() #itertuples(index=False, name='Foo') 

In [ ]:
x = next(stuff)[1]['StudyInstanceUID']


## Set up to call DICOM server


In [1]:
import os
import requests
import io
import pandas as pd
import pydicom
from pathlib import Path
import time
from collections import OrderedDict
from urllib3.filepost import encode_multipart_formdata, choose_boundary

In [2]:
dicom_server_name = "sjbDicom2"
url = f"https://{dicom_server_name}.azurewebsites.net"
url

'https://sjbDicom2.azurewebsites.net'

In [3]:
#from urllib3.filepost import encode_multipart_formdata, choose_boundary
class DicomRequestHelper:
    def __init__(self, url, session):
        self.__url = url
        self.__session = session
    
    
    ########################################
    # All the properties
    
    @property
    def url(self):
        return self.__url
    
    @url.setter
    def url(self, value):
        self.__url=value
    
    @property
    def session(self):
        return self.__session
    
    @session.setter
    def session(self, value):
        self.__session=value
    
    
    ########################################
    # All the misc code
    
    def _encode_multipart_related(self, fields, boundary=None):
        if boundary is None:
            boundary = choose_boundary()

        body, _ = encode_multipart_formdata(fields, boundary)
        content_type = str('multipart/related; boundary=%s' % boundary)

        return body, content_type
    
    
    def _construct_url(self, study_uid, series_uid = None, instance_uid = None, frames = None):        
        #if frames has a value, it's a single item or a list (which must be separated by commas)
        #  also, we must them have a study, series and instance, 
        if frames:
            frame_string = ''
            if type(frames) is list:
                # iterate and separate with commas
                frame_string = ",".join(l)
            else:
                frame_string = frames
            return f'/studies/{study_uid}/series/{series_uid}/instances/{instance_uid}/frames/{frame_string}'
        
        #likewise, if frames=None and instance_uid exists, we must have study and series
        if instance_uid:
            return f'/studies/{study_uid}/series/{series_uid}/instances/{instance_uid}'

        #and so on with series
        if series_uid:
            return f'/studies/{study_uid}/series/{series_uid}'
        
        #must have at least the study
        return f'/studies/{study_uid}'

    def _construct_retrieve_header(self, study_uid, series_uid = None, instance_uid = None, frames = None):
        
        #if frames has a value, it's a single item or a list (which must be separated by commas)
        #  also, we must them have a study, series and instance, 
        if frames:
            return {'Accept':'multipart/related; type="application/octet-stream"; transfer-syntax=*'}
         
        #likewise, if frames=None and instance_uid exists, we must have study and series
        if instance_uid:
            return {'Accept':'application/dicom'}
 
        #and so on with series
        if series_uid:
            return {'Accept':'multipart/related; type="application/dicom"; transfer-syntax=*'}
        
        #must have at least the study
        return {'Accept':'multipart/related; type="application/dicom"; transfer-syntax=*'}

    
    def get_http_response_dict(self, response, is_binary = False, filename=None):
        d = OrderedDict()   # create a new ordered dictionary
        r = response

        # Add the information to the dictionary
        if filename:
            d['fname'] = str(filename)     
        d['method'] = r.request.method
        d['url'] = r.url
        d['path_url'] = r.request.path_url
        d['request_headers'] = str(r.request.headers) 
        if r.request.body:
            d['request_body_trimmed'] = r.request.body[0:150]   

        d['ok'] = r.ok
        d['status_code'] = r.status_code  
        d['reason'] = r.reason
        d['response_headers'] = r.headers
        if not is_binary:
            d['response_text'] = r.text  
            d['apparent_encoding'] = r.apparent_encoding
            d['encoding'] = r.encoding
        d['elapsed_time'] = r.elapsed

        return d  
    
    
    ########################
    # STOW
    
    def store_single_dcm_file(self, filepath):
        
        # Hack. Need to open up and read through file...  Can also do with pydicom
        with open(filepath,'rb') as reader:
            rawfile = reader.read()
        files = {'file': ('dicomfile', rawfile, 'application/dicom')}

        #encode as multipart_related
        body, content_type = self._encode_multipart_related(fields = files)

        headers = {'Accept':'application/dicom+json', "Content-Type":content_type}

        url = f'{self.__url}/studies'
        #response = requests.post(url, body, headers=headers) #, verify=False)
        response = self.session.post(url, body, headers=headers) #, verify=False)

        #return the response object to allow for further processing

        #example usage
        #r = store_single_dcm_file(url,'C:\\githealth\\dicom-samples\\visus.com\\case4\\case4a_002.dcm')
        #print(r.status_code)
        #print(r.request.headers)

        return response
    
    ########################
    # QIDO
    
    def query_all_studies(self):
        
        headers = {'Accept':'application/dicom+json'}

        url = f'{self.__url}/studies'
        response = self.session.get(url, headers=headers) #, verify=False)

        #return the response object to allow for further processing

        #example usage
        #r = store_single_dcm_file(url,'C:\\githealth\\dicom-samples\\visus.com\\case4\\case4a_002.dcm')
        #print(r.status_code)
        #print(r.request.headers)

        return response
    
    def query_all_study_uids(self):
        studies = self.query_all_studies()
        if studies:
            studies_json = studies.json()
            #j = r.json()[0]
            studylist = [x.get('0020000D').get('Value')[0] for x in studies_json]
        
            return studylist
        else:
            return []

    #######################
    # WADO
    
    def retrieve_images(self, study_uid, series_uid = None, instance_uid = None, frames = None):
        url = f'{self.__url}{self._construct_url(study_uid = study_uid, series_uid = series_uid, instance_uid = instance_uid, frames = None)}'
        headers = self._construct_retrieve_header(study_uid = study_uid, series_uid = series_uid, instance_uid = instance_uid, frames = None)
 
        #headers = {'Accept':'application/dicom+json'}
        #headers = {'Accept':'multipart/related; type="application/octet-stream"; transfer-syntax=*'}
        #headers = {'Accept':'multipart/related; type="application/dicom"; transfer-syntax=*'}

        response = self.session.get(url, headers=headers) #, verify=False)
        return response

    def retrieve_metadata(self, study_uid, series_uid = None, instance_uid = None):
        url = f'{self.__url}{self._construct_url(study_uid = study_uid, series_uid = series_uid, instance_uid = instance_uid, frames = None)}/metadata'
        
        headers = {'Accept':'application/dicom+json'}
        print(url)
        response = self.session.get(url, headers=headers) #, verify=False)
        return response
        
    #######################
    # DELETE
    
    def delete_study(self, study_uid):
        headers = {'Accept':'application/dicom+json'}
        #headers = {'Accept':'anything/at+all'}

        url = f'{self.__url}/studies/{study_uid}'
        response = self.session.delete(url, headers=headers) #, verify=False)

        #return the response object to allow for further processing

        #example usage
        #r = store_single_dcm_file(url,'C:\\githealth\\dicom-samples\\visus.com\\case4\\case4a_002.dcm')
        #print(r.status_code)
        #print(r.request.headers)

        return response
    
    def delete_studies(self, study_uids):
        return [self.delete_study(study) for study in study_uids]
            
    
    def mangle_dicom_with_new_uids(self,datasets, num_studies = None, num_series=None):
        
        #if datasets isn't a list (a single one, perhaps?), make it a list
        if type(datasets) is not list:
            y = []
            y.append(datasets)
            datasets = y
        
        # if num_studies and num_series aren't passed, just make everything unique
        if not num_studies and not num_series:
            num_studies = len(datasets)
            num_series = len(datasets)
        
        pass
    
    def download_studies_to_pydicom(self,num_studies=50):
        '''Retrieve num_studies from the server and convert to pydicom datasets'''
        study_uids = self.query_all_study_uids() #get all the studies, up to max server will send back
        study_uids = study_uids[:num_studies] # trim to only the first num_studies
        
        # download all the studies
        pydicom_studies = []
        for uid in study_uids:
            # get the study
            study = self.retrieve_images()
            
        
        
        

In [4]:
client = requests.session()
x = DicomRequestHelper(url, client)

In [5]:
x.url

'https://sjbDicom2.azurewebsites.net'

## Test WADO

In [ ]:
# find a study
studies = x.query_all_study_uids()
print(len(studies))

In [ ]:
print(studies[0])

In [ ]:
#z = [x.retrieve_images(study) for study in studies]
#z

In [ ]:
#yy = x.retrieve_metadata('1.2.276.0.50.192168001099.9483698.14547392.4')
y = x.retrieve_images('1.2.276.0.50.192168001099.9483698.14547392.4')

In [ ]:
sho = x.get_http_response_dict(y, is_binary=True)
sho

## Test QIDO

In [20]:
r = x.query_all_studies()

In [21]:
d = x.get_http_response_dict(r)
d

OrderedDict([('method', 'GET'),
             ('url', 'https://sjbdicom2.azurewebsites.net/studies'),
             ('path_url', '/studies'),
             ('request_headers',
              "{'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/dicom+json', 'Connection': 'keep-alive'}"),
             ('ok', True),
             ('status_code', 200),
             ('reason', 'OK'),
             ('response_headers',
              {'Content-Length': '5892', 'Content-Type': 'application/dicom+json; charset=utf-8', 'Server': 'Microsoft-IIS/10.0', 'Request-Context': 'appId=cid-v1:be5422fb-ccaa-4f0e-9a84-86cf4c3f0089', 'X-Content-Type-Options': 'nosniff', 'X-Powered-By': 'ASP.NET', 'Date': 'Wed, 13 May 2020 18:29:17 GMT'}),
             ('response_text',
              '[{"00080020":{"vr":"DA","Value":["19941013"]},"00080030":{"vr":"TM","Value":["141917"]},"00080050":{"vr":"SH"},"00080090":{"vr":"PN"},"00081030":{"vr":"LO"},"00100010":{"vr":"PN","Value"

In [14]:
l = x.query_all_study_uids()

In [15]:

l

['1.3.12.2.1107.5.4.3.123456789012345.19950922.121803.6',
 '1.2.276.0.50.192168001099.9483698.14547392.4',
 '1.2.276.0.50.192168001099.9140875.14547392.277',
 '1.2.276.0.50.192168001099.9140875.14547392.4',
 '1.2.276.0.50.192168001099.8829267.14547392.4',
 '1.2.276.0.50.192168001092.11517584.14547392.4',
 '1.2.276.0.50.192168001099.8687553.14547392.4',
 '1.2.276.0.50.192168001092.11156604.14547392.4',
 '1.2.276.0.50.192168001099.8252157.14547392.4',
 '1.2.276.0.50.192168001099.7810872.14547392.270']

## Test STOW

In [ ]:
### SAVE!!!!  Causes a 500 error on server
#r = x.store_single_dcm_file('C:\\githealth\\dicom-samples\\visus.com\\case4\\case4a_002.dcm')
#r = x.store_single_dcm_file("C:\\githealth\\dicom-samples\\dicomlibrary.com\\series-000001\\image-000360.dcm") #500
#r = x.store_single_dcm_file("C:\\githealth\\dicom-samples\\dicomlibrary.com\\series-000001\\image-000001.dcm") #500
#r = x.store_single_dcm_file("C:\\githealth\\dicom-samples\\barre.dev\\MR-MONO2-8-16x-heart.dcm")  #500

r = x.store_single_dcm_file("C:\\githealth\\dicom-samples\\rubomedica.com\\0002.dcm") 


In [6]:
filename = "C:\\githealth\\dicom-samples\\rubomedica.com\\0002.dcm"
r = x.store_single_dcm_file(filename) 

In [7]:
r

<Response [409]>

In [ ]:
d = x.get_http_response_dict(r, filename)

In [ ]:
d

## Test DELETE

In [ ]:
uid = '1.3.12.2.1107.5.4.3.4975316777216.19951114.94101.16'

In [ ]:
r = x.delete_study(uid)

In [ ]:
y = x.get_http_response_dict(r)

In [ ]:
st = x.query_all_study_uids()

## Prep for Load Test

In [ ]:
studies = x.query_all_study_uids()

In [ ]:
studies

In [ ]:
meta = [x.retrieve_metadata(s) for s in studies]

In [ ]:
jsons = [m.json() for m in meta]   
jsons


In [ ]:
# 00080018 = SOPInstanceUID
# 0020000D = StudyInstanceUID
# 0020000E = SeriesInstanceUID

In [ ]:
   
instance_uids = [j[0].get('00080018').get('Value')[0] for j in jsons]
series_uids = [j[0].get('0020000E').get('Value')[0] for j in jsons]
study_uids = [j[0].get('0020000D').get('Value')[0] for j in jsons]
uids = [(j[0].get('0020000D').get('Value')[0],j[0].get('0020000E').get('Value')[0],j[0].get('00080018').get('Value')[0]) for j in jsons]

In [ ]:
uids # this is a tuple with 3 uids in order (study, series, instance)

In [ ]:
# Get a list of instance responses
instances = [x.retrieve_metadata(st,se,ins) for st,se,ins in uids]

In [ ]:
st,se,ins = uids[0]
print(st)
print(se)
print(ins)
inst = x.retrieve_images(st,se,ins)
inst2 = x.retrieve_metadata(st,se,ins)

In [ ]:
inst.content


In [ ]:
from io import BytesIO
from pydicom import dcmread, dcmwrite
from pydicom.filebase import DicomFileLike

dataset = dcmread(BytesIO(inst.content), force=True)

In [ ]:
dataset